In [8]:
import pandas as pd
import tensorflow as tf
import pickle
import numpy as np

In [2]:
X = pickle.load(open('../Data/X_test', 'rb'))
y = pickle.load(open('../Data/y_test', 'rb'))

In [3]:
print(X.shape, y.shape)

(2, 1736, 978) (1736,)


In [4]:
tf.reset_default_graph()

n_layers = 10
n_inputs = 16
n_classes = 16
learning_rate = 0.05

input1 = tf.placeholder(dtype=tf.float32, shape=[None, 978])
input2 = tf.placeholder(dtype=tf.float32, shape=[None, 978])
orig_1 = input1
orig_2 = input2

labels = tf.placeholder(dtype=tf.float32, shape=(None))

for a in range(n_layers):
    a_layer = tf.layers.dense(input1, n_inputs, 'selu', name='layer'+str(a))
    input1 = tf.concat([input1, a_layer], 1, name='a_concatenation'+str(a))
    
for a in range(n_layers):
    b_layer = tf.layers.dense(input2, n_inputs, 'selu', name='layer'+str(a), reuse=True)
    input2 = tf.concat([input2, b_layer], 1, name='b_concatenation'+str(a))
    
emb1 = tf.nn.l2_normalize(tf.layers.dense(input1, 16, None), axis=1)
emb2 = tf.nn.l2_normalize(tf.layers.dense(input2, n_classes, None), axis=1)

distance = tf.reduce_sum(tf.multiply(emb1, emb2),axis=1)
diff = tf.subtract(distance, labels)
loss = tf.reduce_mean(tf.multiply(diff, diff))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
with tf.Session() as session:  
    tf.initialize_all_variables().run()
    print("Initialized")
    
#     feed_dict={orig_1:X[0], orig_2:X[1], labels:y}
    
    feed_dict={orig_1:X[0], orig_2:X[1], labels:y}
    
    for a in range(100):
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        print(l)
    d = session.run([distance], feed_dict=feed_dict)
    feed_dict={orig_1:X[0], orig_2:X[0], labels:y}
    e1, e2 = session.run([emb1, emb2], feed_dict=feed_dict)
    print(e1, e2)
#     print(d)
    
    tf.summary.FileWriter('./logs', session.graph)

Initialized
0.5662695
0.6179323
0.29235843
0.47624302
0.2522788
0.27782902
0.29336467
0.28379706
0.26597592
0.2521252
0.24890196
0.25430688
0.26032406
0.26113468
0.25715828
0.2520977
0.24885158
0.24819426
0.2492441
0.25095236
0.25217706
0.25241563
0.2517209
0.25038013
0.2490133
0.24800435
0.24755768
0.24770492
0.248217
0.24871801
0.24893716
0.24881451
0.24833773
0.24774599
0.24720891
0.24690145
0.24685293
0.2470116
0.2469815
0.24693035
0.24672814
0.24646413
0.24609178
0.24572235
0.2453518
0.24509245
0.24497983
0.24475577
0.24426551
0.24366426
0.24295829
0.24216014
0.24081555
0.23866716
0.2346594
0.22989468
0.22706749
0.23304485
0.22700159
0.21622679
0.20776212
0.19889493
0.20698734
0.20476706
0.21239333
0.1951125
0.18523645
0.1805501
0.1671569
0.16544965
0.15847065
0.14923044
0.14112894
0.13307284
0.12520048
0.119859785
0.11356616
0.108463906
0.1046536
0.10158211
0.09811846
0.09542283
0.092333965
0.08975617
0.08746481
0.08464741
0.08284614
0.08092949
0.07906379
0.077311926
0.07570353
0

In [15]:
xnew1 = []
xnew2 = []
ynew = []
for a in range(len(y)):
    if y[a]==0:
        xnew1.append(X[0][a])
        xnew2.append(X[0][a])
        ynew.append(y[a])
xnew1 = np.asarray(xnew1)
xnew2 = np.asarray(xnew2)
ynew = np.asarray(ynew)

In [16]:
xnew1.shape

(868, 978)

In [17]:
ynew.shape

(868,)

In [18]:
e1[0]

array([ 0.23216619, -0.2976543 , -0.2785324 ,  0.19524847,  0.21490829,
        0.16193542,  0.4253295 ,  0.27673236, -0.14786448, -0.16926019,
       -0.24436551, -0.23750608,  0.01472671,  0.06601524, -0.41277626,
        0.26515725], dtype=float32)

In [19]:
e2[0]

array([ 0.16106819,  0.10732867,  0.26146308, -0.26907846, -0.26455677,
        0.35873795,  0.13699406, -0.15860845, -0.34518462, -0.44496343,
       -0.25078627,  0.23644447, -0.02990031, -0.3010375 , -0.17595555,
       -0.14401956], dtype=float32)